# Data Exploration, Pre-Processing, and Loading

## 1 Investigation

In [1]:
import os
import gzip
import torch
import nibabel as nib

# Move working dir to access src
os.chdir("../")

def read_image(file_path):
    nii_img = nib.load(file_path)
    image_data = nii_img.get_fdata()
    header = nii_img.header
    return image_data, header

In [2]:
image_data, _ = read_image('data_dev/train/images/RibFrac422-image.nii.gz') 
label_data, header = read_image('data_dev/train/labels/RibFrac422-label.nii.gz')

In [3]:
# Add batch and channel dimensions
image_data = torch.tensor(image_data).unsqueeze(0).unsqueeze(0).float()

In [5]:
from src.model.models import UNet3D

model = UNet3D(1, 3)

data = image_data[:, :112, :112, :112]
y = model(data)
y.shape

torch.Size([1, 3, 112, 112, 320])